CountryStatistics 클래스
__clean_numeric_data(value: str): 문자열을 입력으로 받아 쉼표, 달러 기호, 백분율 기호를 제거하고 숫자 값을 float으로 반환합니다. 입력을 변환할 수 없으면 0.0을 반환합니다.
힌트: 이 함수는 쉼표나 달러 기호가 포함된 CSV 파일의 숫자 데이터를 처리하는 데 유용합니다. 빈 문자열이나 숫자가 아닌 값을 처리할 수 있도록 예외 처리에 신경 쓰세요.

__load_data(filename: str): CSV 파일에서 국가 데이터를 로드하고 데이터를 파싱하여 Country 객체를 생성합니다. 정리된 데이터는 self.countries 리스트에 저장됩니다. 파일 로드 완료 후 현재 시간을 self.load_time에 저장합니다.
힌트: csv.reader를 사용하여 CSV 파일을 한 줄씩 읽습니다. 첫 번째 행은 헤더이므로 건너뜁니다. 모든 숫자 데이터 필드에 대해 __clean_numeric_data를 호출하여 데이터를 저장하기 전에 정확하게 정리된 데이터를 확인하세요. GDP, 기대 수명, 인구 등의 열 인덱스를 정확하게 매핑하세요.

top_5_gdp(): GDP가 가장 높은 상위 5개 국가의 이름과 쉼표로 형식화된 GDP 값을 포함한 튜플 목록을 반환합니다.
힌트: 국가들을 gdp 속성 기준으로 내림차순 정렬하세요. 슬라이싱 ([:5])을 사용하여 상위 5개 항목을 선택하세요. 국가 이름과 형식화된 GDP 값을 반환하세요.

top_5_life_expectancy(): 기대 수명이 가장 긴 상위 5개 국가의 이름과 그들의 기대 수명 값을 포함한 튜플 목록을 반환합니다.
힌트: 국가들을 기대 수명(life_expectancy) 기준으로 내림차순 정렬하세요. 슬라이싱 ([:5])을 사용하여 상위 5개 항목을 추출하세요. 기대 수명 값은 형식화할 필요가 없으므로 있는 그대로 반환하세요.

countries_by_density(): 인구 밀도가 가장 높은 상위 5개 국가의 이름과 인구 밀도를 반환합니다.
힌트: 국가들을 인구 밀도(density) 속성 기준으로 내림차순 정렬하세요. 슬라이싱 ([:5])을 사용하여 상위 5개 항목을 선택하세요. 인구 밀도 값은 정수이므로 형식화할 필요가 없습니다.

get_data_load_time(format_str=None): 데이터 로드 시간을 반환하며, format_str로 사용자 지정 포맷을 지원합니다. 데이터가 아직 로드되지 않았다면 None을 반환합니다.
힌트: strftime 메서드를 이용해 봅니다.

plot_geo_scatter(self): 위도와 경도 데이터를 사용하여 지리적 산점도를 그립니다.
힌트: plotly.express.scatter_geo 메서드를 이용해 봅니다. (16.2 이후 도전하기)

In [7]:
import csv
from dataclasses import dataclass
import pandas as pd
import plotly.express as px
import time
import datetime as dt


@dataclass
class Country:
    name: str
    gdp: float
    life_expectancy: float
    density: float
    lat: float
    lon: float


class CountryStatistics:
    def __init__(self):
        self.countries = []
        self.load_time = None

    def clean_numeric_data(self, value: str):
        try:
            value = value.replace(",", "").replace("$", "").replace("%", "")
            return float(value)
        except:
            return 0.0

    def load_data(self, filename: str):
        with open(filename, encoding="UTF-8") as file:
            reader = csv.reader(file)
            header_row = next(reader)
            for row in reader:
                Country = {
                    "name": row[0],
                    "gdp": self.clean_numeric_data(row[16]),
                    "life_expectancy": self.clean_numeric_data(row[21]),
                    "density": self.clean_numeric_data(row[1]),
                    "lat": self.clean_numeric_data(row[33]),
                    "lon": self.clean_numeric_data(row[34]),
                }
                self.countries.append(Country)
        self.load_time = time.time()
        return self.countries

    def top_5_gdp(self):
        top_countries = sorted(self.countries, key=lambda x: x["gdp"], reverse=True)[:5]
        return [
            (country["name"], f"${country['gdp']:,.2f}") for country in top_countries
        ]

    def top_5_life_expectancy(self):
        top_countries = sorted(
            self.countries, key=lambda x: x["life_expectancy"], reverse=True
        )[:5]
        return [
            (country["name"], country["life_expectancy"]) for country in top_countries
        ]

    def countries_by_density(self):
        top_countries = sorted(
            self.countries, key=lambda x: x["density"], reverse=True
        )[:5]
        return [(country["name"], country["density"]) for country in top_countries]

    def get_data_load_time(self, format_str=None):
        if self.load_time == None:
            return None
        if format_str:
            return dt.datetime.fromtimestamp(self.load_time).strftime(format_str)

    def plot_geo_scatter(self):
        fig = px.scatter_geo(
            self.countries,
            lat="lat",
            lon="lon",
            hover_name="name",  # 마우스 올리면 국가명 표시
            title="World Country Geo Scatter",
        )
        fig.show()


c = CountryStatistics()
print(c.load_data("world-data-2023.csv"))
print(c.top_5_gdp())
print(c.top_5_life_expectancy())
print(c.countries_by_density())
print(c.get_data_load_time("%Y-%m-%d %H:%M:%S"))
c.plot_geo_scatter()

[{'name': 'Afghanistan', 'gdp': 19101353833.0, 'life_expectancy': 64.5, 'density': 60.0, 'lat': 33.93911, 'lon': 67.709953}, {'name': 'Albania', 'gdp': 15278077447.0, 'life_expectancy': 78.5, 'density': 105.0, 'lat': 41.153332, 'lon': 20.168331}, {'name': 'Algeria', 'gdp': 169988236398.0, 'life_expectancy': 76.7, 'density': 18.0, 'lat': 28.033886, 'lon': 1.659626}, {'name': 'Andorra', 'gdp': 3154057987.0, 'life_expectancy': 0.0, 'density': 164.0, 'lat': 42.506285, 'lon': 1.521801}, {'name': 'Angola', 'gdp': 94635415870.0, 'life_expectancy': 60.8, 'density': 26.0, 'lat': -11.202692, 'lon': 17.873887}, {'name': 'Antigua and Barbuda', 'gdp': 1727759259.0, 'life_expectancy': 76.9, 'density': 223.0, 'lat': 17.060816, 'lon': -61.796428}, {'name': 'Argentina', 'gdp': 449663446954.0, 'life_expectancy': 76.5, 'density': 17.0, 'lat': -38.416097, 'lon': -63.616672}, {'name': 'Armenia', 'gdp': 13672802158.0, 'life_expectancy': 74.9, 'density': 104.0, 'lat': 40.069099, 'lon': 45.038189}, {'name': '